# Chunk Finder for Meaningful Forms

## Imports 

In [1]:
import collections
from pathlib import Path

import numpy as np
import pandas as pd
import spacy
from spacy import displacy
from spacy.matcher import PhraseMatcher
import textacy

pd.set_option("display.max_rows", 600)
pd.set_option("display.max_columns", 500)
pd.set_option("max_colwidth", 400)


spacy.require_gpu()
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf")


In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import pandas as pd

embedder = SentenceTransformer('all-mpnet-base-v2')
pd_cluster = pd.read_csv('D:\\NLP\\Frame_NLP\\archive\\Sent_Cluster\\labeled_clusters99.csv')

## Small Cluster Sentence Similarity Checker

In [ ]:
# Corpus with example sentences
corpus_sents = pd_cluster[pd_cluster['category']=='reports_case_coronavirus_tally']

corpus_sents.head()

In [ ]:
corpus = corpus_sents['Processed_Title'].tolist()
corpus[:20]

In [ ]:
corpus_embeddings = embedder.encode(corpus)

In [ ]:
# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(
    [corpus_embeddings[0]],
    corpus_embeddings[1:]
)


## 1229 Categories Clustering

In [22]:
pd_category = pd.read_csv('D:\\NLP\\Frame_NLP\\archive\\Sent_Cluster\\label_freq.csv')

In [23]:
column_to_remove = ['label']
corpus_categories = pd_category[column_to_remove].replace({'_':' '}, regex=True)
corpus_categories[:10]

,label
0,coronavirus coronavirus case
1,reports case coronavirus death
2,confirms case coronavirus
3,reports case coronavirus
4,coronavirus market
5,race vaccine coronavirus
6,coronavirus pinch holiday
7,coronavirus lockdown
8,coronavirus transmission glance
9,canceled fear coronavirus concern


In [25]:
cat_corpus = corpus_categories['label'].tolist()
cat_corpus[:10]

['coronavirus coronavirus case',
 'reports case coronavirus death',
 'confirms case coronavirus',
 'reports case coronavirus',
 'coronavirus market',
 'race vaccine coronavirus',
 'coronavirus pinch holiday',
 'coronavirus lockdown',
 'coronavirus transmission glance',
 'canceled fear coronavirus concern']

In [26]:
corpus_embeddings = embedder.encode(cat_corpus)

In [41]:
# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(cat_corpus[sentence_id])

f = open('D://NLP//Frame_NLP//archive//Sent_Cluster//coronavirus_Frames.txt', 'w', encoding='utf-8')
#f = open('D://NLP//Frame_NLP//archive//Sent_Cluster//coronavirus_frames.csv', 'w', encoding='utf-8')

for i, cluster in clustered_sentences.items():

    print("Cluster {} \t\t {}".format(i+1, cluster), file=f)
    #print("Cluster %(n)s \t\t %(b)s" % {'n': i+1, 'b': cluster}, file=f)

f.close()

In [43]:
category_clusters = pd.read_csv('D://NLP//Frame_NLP//archive//Sent_Cluster//coronavirus_Frames.txt', sep='\t\t', encoding='utf-8')
category_clusters.head()
category_clusters.columns.values[0] = "Cluster Number"
category_clusters.columns.values[1] = "List of Frame Aspects"

display(category_clusters)

,Cluster 90,"['coronavirus coronavirus case', 'cases case coronavirus', 'came infections coronavirus case', 'coronavirus epidemic case', 'coronavirus thrombus case', 'case case coronavirus', 'suggests infection coronavirus case', 'case case coronavirus health', 'case case coronavirus 1st', 'cases case coronavirus ministry', 'isolated member coronavirus case', 'coronavirus case', 'crosses cases coronavirus', 'turns coronavirus case', 'sees case coronavirus', 'case coronavirus', 'cases coronavirus case', 'cases coronavirus infection', 'cases coronavirus patient', 'cases 10,000 coronavirus case', 'arrived cases coronavirus carrier', 'cases coronavirus', 'cases coronavirus investigation']"
0,Cluster 86,"['reports case coronavirus death', 'dies victim coronavirus doctor', 'reports death coronavirus', 'confirms death coronavirus', 'reaches official death coronavirus', 'reports death coronavirus doctor', 'announces death case coronavirus', 'dies covid person coronavirus', 'confirms case coronavirus death', 'reports death coronavirus case', 'reports death coronavirus person', 'cleared death coro..."
1,Cluster 60,"['confirms case coronavirus', 'confirms case coronavirus governor', 'confirmed case coronavirus', 'confirmed coronavirus case', 'confirms case coronavirus police', 'confirms case coronavirus chief', 'confirms case coronavirus month', 'confirms case coronavirus report', 'confirms case president coronavirus', 'confirmed case coronavirus virus', 'confirms case coronavirus minister', 'confirms ca..."
2,Cluster 14,"['reports case coronavirus', 'reports loss case coronavirus', 'reports spike case coronavirus', 'reports case coronavirus tally', 'newsletter hospitalizations coronavirus outbreak', 'reported news coronavirus case', 'reported case coronavirus', 'reports rise coronavirus death', 'records death coronavirus patient', 'reports case coronavirus hospital', 'records case coronavirus 13th', 'records ..."
3,Cluster 78,"['coronavirus market', 'coronavirus economy', 'coronavirus growth economy outbreak', 'coronavirus recession economy', 'says depression coronavirus economy', 'warns recession coronavirus', 'crisis economy coronavirus lockdown', 'coronavirus potential case economy', 'lawmaker coronavirus economy factor']"
4,Cluster 38,"['race vaccine coronavirus', 'coronavirus authoritarianism opinion', 'speak negros coronavirus racism', 'cancels race coronavirus outbreak', 'coronavirus minority', 'races surge case coronavirus', 'discriminate coronavirus supremacist', 'infected racism coronavirus opinion', 'use coronavirus racism excuse', 'stop spread racism coronavirus']"


In [108]:
df = pd.DataFrame()

cluster_dic = {}

for i, cluster in clustered_sentences.items():
    cluster_dic = {**cluster_dic, **{"Cluster {}".format(i+1): cluster}}
    
df = pd.DataFrame.from_dict(cluster_dic, orient = 'index')
df = df.T
df = df.replace({' ':'_'}, regex=True)

df.to_csv('D://NLP//Frame_NLP//archive//Sent_Cluster//coronavirus_Frames.csv')

In [110]:
Frames = pd.read_csv('D://NLP//Frame_NLP//archive//Sent_Cluster//coronavirus_Frames.csv', index_col=0)
Frames.head(50)

,Cluster 90,Cluster 86,Cluster 60,Cluster 14,Cluster 78,Cluster 38,Cluster 19,Cluster 98,Cluster 7,Cluster 27,Cluster 56,Cluster 63,Cluster 22,Cluster 12,Cluster 46,Cluster 26,Cluster 8,Cluster 37,Cluster 68,Cluster 49,Cluster 71,Cluster 33,Cluster 29,Cluster 100,Cluster 5,Cluster 24,Cluster 28,Cluster 84,Cluster 85,Cluster 16,Cluster 54,Cluster 13,Cluster 53,Cluster 25,Cluster 15,Cluster 74,Cluster 20,Cluster 82,Cluster 94,Cluster 42,Cluster 51,Cluster 59,Cluster 1,Cluster 99,Cluster 91,Cluster 58,Cluster 72,Cluster 43,Cluster 93,Cluster 101,Cluster 3,Cluster 39,Cluster 30,Cluster 31,Cluster 57,Cluster 52,Cluster 79,Cluster 64,Cluster 73,Cluster 66,Cluster 17,Cluster 76,Cluster 96,Cluster 32,Cluster 45,Cluster 67,Cluster 77,Cluster 23,Cluster 50,Cluster 88,Cluster 36,Cluster 70,Cluster 34,Cluster 48,Cluster 10,Cluster 18,Cluster 69,Cluster 35,Cluster 83,Cluster 21,Cluster 44,Cluster 92,Cluster 80,Cluster 55,Cluster 47,Cluster 6,Cluster 89,Cluster 2,Cluster 61,Cluster 95,Cluster 41,Cluster 40,Cluster 4,Cluster 81,Cluster 97,Cluster 11,Cluster 75,Cluster 62,Cluster 87,Cluster 9,Cluster 65
0,coronavirus_coronavirus_case,reports_case_coronavirus_death,confirms_case_coronavirus,reports_case_coronavirus,coronavirus_market,race_vaccine_coronavirus,coronavirus_pinch_holiday,coronavirus_lockdown,coronavirus_transmission_glance,canceled_fear_coronavirus_concern,closes_school_coronavirus,urges_people_coronavirus_scare,coronavirus_coronavirus_update_look,test_immunity_coronavirus_antibody,coronavirus_restriction_patient,coronavirus_event_sport,coronavirus_coronavirus_cure,looms_disinformation_coronavirus_misinformation,declares_emergency_coronavirus,coronavirus_nightmare_prayer,got_case_coronavirus_jail,system_result_coronavirus_state,surges_case_coronavirus,coronavirus_ward_hospital,coronavirus_coronavirus_player,says_risk_coronavirus_airline,reaches_112_coronavirus_death,rises_coronavirus_death,coronavirus_thing_home_care,coronavirus_development_nrl,enforce_measure_coronavirus_government,changing_disruption_coronavirus_business,passes_bill_coronavirus_relief,recovered_recovery_coronavirus_case,says_research_coronavirus_hacker,coronavirus_coronavirus_quarantine,closes_border_coronavirus,coronavirus_positive_ship_cruise,plunge_sell_market_coronavirus,coronavirus_dad_librarian,tests_negative_coronavirus_doctor,closed_facility_coronavirus_tourist,spread_coronavirus_dog_cat,coronavirus_update_case,approves_test_coronavirus,voted_hotspot_coronavirus_case,tests_positive_coronavirus_symptom,coronavirus_life_outbreak,says_return_coronavirus_job,tests_positive_owner_knick,coronavirus_mass_church,worried_shot_coronavirus_flu,pence_member_coronavirus_fight,latest_cases_death_case,"coronavirus_6,000_case",coronavirus_ventilator,think_misinformation_coronavirus_beer,says_coronavirus_patient,extend_measure_coronavirus_case,coronavirus_mask_pandemic,halts_warranty_coronavirus_day,handles_shopper_coronavirus_staff,grow_coronavirus_scientist_lab,coronavirus_coronavirus_risk,coronavirus_movie,spend_fund_coronavirus_response,ready_stimulus_coronavirus_temper,seek_hand_coronavirus_govts,signs_package_coronavirus_emergency,builds_awareness_coronavirus,updates_beaches_coronavirus_pandemic,tells_china_coronavirus_outbreak,report_app_coronavirus,confirmed_amvca_case_isolation,coronavirus_self_victim,pizzeria_customer_coronavirus_dining,declares_curfew_coronavirus_lockdown,says_response_whos_curve,coronavirus_uk_travel_ban,play_role_woman_parent,podcast_podcast_time_houseparty,aged_coronavirus_woman,hospitalized_president_wife,dies_ambassador_leader_aide,protocol_plan_instruction_disease,suffer_blow_etfs_opportunity,timeline_sport_response,rises_9692_death_toll,updates_update_world,underlines_need_effort_govt,tightens_case_tourist_emergency,desantis_millions_case_state,announces_recovery_patient_death,rises_number_case,reports_spike_view_industry,help_game_developer,fall_price_oil,update_dw_birthday,waives_fee_coronavirus_consumer,bet_opinion_cure,news_ship_cru

In [92]:
Frames.shape

(53, 101)

## Frame Sub-Frame Corpus Maker

In [118]:
pd_all = pd.read_csv('D:\\NLP\\Frame_NLP\\archive\\Sent_Cluster\\labeled_clusters99.csv')
pd_all.head()

,Processed_Title,label_st1,label,category
0,"US coronavirus death toll rises to 14, most cases in Washington",85,reports_case_coronavirus_death,rises_37_coronavirus_death
1,"Iran rejects U.S. offer for coronavirus aid, cites conspiracy theory",17,recover_coronavirus_death,refuses_offer_coronavirus_conspiracy
2,TSA is making 3 changes to stop coronavirus from spreading on planes,-1,coronavirus_coronavirus_pandemic,expands_change_airport_coronavirus
3,NBA Players to Take Pay Cut Amidst Coronavirus Crisis,64,coronavirus_coronavirus_player,preparing_cut_coronavirus_game
4,"Coronavirus Cases confirmed in North Korea, say officials",21,vote_kit_coronavirus_case,coronavirus_exercise_case_week


In [114]:
category_clusters.iloc[0,1]

" ['reports case coronavirus death', 'dies victim coronavirus doctor', 'reports death coronavirus', 'confirms death coronavirus', 'reaches official death coronavirus', 'reports death coronavirus doctor', 'announces death case coronavirus', 'dies covid person coronavirus', 'confirms case coronavirus death', 'reports death coronavirus case', 'reports death coronavirus person', 'cleared death coronavirus news', 'confirmed coronavirus death', 'announced death coronavirus', 'dies coronavirus news', 'updated case coronavirus death', 'announces death coronavirus patient', 'stories talk coronavirus death', 'cases victim coronavirus death', 'reports death coronavirus life', 'confirms death coronavirus official', 'confirm death coronavirus health', 'dies coronavirus patient', 'confirms death coronavirus patient', 'died victim coronavirus patient']"

In [119]:
Frames.head(30)

,Cluster 90,Cluster 86,Cluster 60,Cluster 14,Cluster 78,Cluster 38,Cluster 19,Cluster 98,Cluster 7,Cluster 27,Cluster 56,Cluster 63,Cluster 22,Cluster 12,Cluster 46,Cluster 26,Cluster 8,Cluster 37,Cluster 68,Cluster 49,Cluster 71,Cluster 33,Cluster 29,Cluster 100,Cluster 5,Cluster 24,Cluster 28,Cluster 84,Cluster 85,Cluster 16,Cluster 54,Cluster 13,Cluster 53,Cluster 25,Cluster 15,Cluster 74,Cluster 20,Cluster 82,Cluster 94,Cluster 42,Cluster 51,Cluster 59,Cluster 1,Cluster 99,Cluster 91,Cluster 58,Cluster 72,Cluster 43,Cluster 93,Cluster 101,Cluster 3,Cluster 39,Cluster 30,Cluster 31,Cluster 57,Cluster 52,Cluster 79,Cluster 64,Cluster 73,Cluster 66,Cluster 17,Cluster 76,Cluster 96,Cluster 32,Cluster 45,Cluster 67,Cluster 77,Cluster 23,Cluster 50,Cluster 88,Cluster 36,Cluster 70,Cluster 34,Cluster 48,Cluster 10,Cluster 18,Cluster 69,Cluster 35,Cluster 83,Cluster 21,Cluster 44,Cluster 92,Cluster 80,Cluster 55,Cluster 47,Cluster 6,Cluster 89,Cluster 2,Cluster 61,Cluster 95,Cluster 41,Cluster 40,Cluster 4,Cluster 81,Cluster 97,Cluster 11,Cluster 75,Cluster 62,Cluster 87,Cluster 9,Cluster 65
0,coronavirus_coronavirus_case,reports_case_coronavirus_death,confirms_case_coronavirus,reports_case_coronavirus,coronavirus_market,race_vaccine_coronavirus,coronavirus_pinch_holiday,coronavirus_lockdown,coronavirus_transmission_glance,canceled_fear_coronavirus_concern,closes_school_coronavirus,urges_people_coronavirus_scare,coronavirus_coronavirus_update_look,test_immunity_coronavirus_antibody,coronavirus_restriction_patient,coronavirus_event_sport,coronavirus_coronavirus_cure,looms_disinformation_coronavirus_misinformation,declares_emergency_coronavirus,coronavirus_nightmare_prayer,got_case_coronavirus_jail,system_result_coronavirus_state,surges_case_coronavirus,coronavirus_ward_hospital,coronavirus_coronavirus_player,says_risk_coronavirus_airline,reaches_112_coronavirus_death,rises_coronavirus_death,coronavirus_thing_home_care,coronavirus_development_nrl,enforce_measure_coronavirus_government,changing_disruption_coronavirus_business,passes_bill_coronavirus_relief,recovered_recovery_coronavirus_case,says_research_coronavirus_hacker,coronavirus_coronavirus_quarantine,closes_border_coronavirus,coronavirus_positive_ship_cruise,plunge_sell_market_coronavirus,coronavirus_dad_librarian,tests_negative_coronavirus_doctor,closed_facility_coronavirus_tourist,spread_coronavirus_dog_cat,coronavirus_update_case,approves_test_coronavirus,voted_hotspot_coronavirus_case,tests_positive_coronavirus_symptom,coronavirus_life_outbreak,says_return_coronavirus_job,tests_positive_owner_knick,coronavirus_mass_church,worried_shot_coronavirus_flu,pence_member_coronavirus_fight,latest_cases_death_case,"coronavirus_6,000_case",coronavirus_ventilator,think_misinformation_coronavirus_beer,says_coronavirus_patient,extend_measure_coronavirus_case,coronavirus_mask_pandemic,halts_warranty_coronavirus_day,handles_shopper_coronavirus_staff,grow_coronavirus_scientist_lab,coronavirus_coronavirus_risk,coronavirus_movie,spend_fund_coronavirus_response,ready_stimulus_coronavirus_temper,seek_hand_coronavirus_govts,signs_package_coronavirus_emergency,builds_awareness_coronavirus,updates_beaches_coronavirus_pandemic,tells_china_coronavirus_outbreak,report_app_coronavirus,confirmed_amvca_case_isolation,coronavirus_self_victim,pizzeria_customer_coronavirus_dining,declares_curfew_coronavirus_lockdown,says_response_whos_curve,coronavirus_uk_travel_ban,play_role_woman_parent,podcast_podcast_time_houseparty,aged_coronavirus_woman,hospitalized_president_wife,dies_ambassador_leader_aide,protocol_plan_instruction_disease,suffer_blow_etfs_opportunity,timeline_sport_response,rises_9692_death_toll,updates_update_world,underlines_need_effort_govt,tightens_case_tourist_emergency,desantis_millions_case_state,announces_recovery_patient_death,rises_number_case,reports_spike_view_industry,help_game_developer,fall_price_oil,update_dw_birthday,waives_fee_coronavirus_consumer,bet_opinion_cure,news_ship_cru

In [128]:
label_list_86 = Frames['Cluster 86'].tolist()
label_list_86[:50]

['reports_case_coronavirus_death',
 'dies_victim_coronavirus_doctor',
 'reports_death_coronavirus',
 'confirms_death_coronavirus',
 'reaches_official_death_coronavirus',
 'reports_death_coronavirus_doctor',
 'announces_death_case_coronavirus',
 'dies_covid_person_coronavirus',
 'confirms_case_coronavirus_death',
 'reports_death_coronavirus_case',
 'reports_death_coronavirus_person',
 'cleared_death_coronavirus_news',
 'confirmed_coronavirus_death',
 'announced_death_coronavirus',
 'dies_coronavirus_news',
 'updated_case_coronavirus_death',
 'announces_death_coronavirus_patient',
 'stories_talk_coronavirus_death',
 'cases_victim_coronavirus_death',
 'reports_death_coronavirus_life',
 'confirms_death_coronavirus_official',
 'confirm_death_coronavirus_health',
 'dies_coronavirus_patient',
 'confirms_death_coronavirus_patient',
 'died_victim_coronavirus_patient',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 n

In [141]:
cluster_corpus = []

for i in label_list_86:
    if i != 'nan':
        pd_old = pd_all[pd_all['category']=='{}'.format(i)]['Processed_Title'].tolist()
        pd_new = [x for x in pd_old if x != 'nan']
        for item in pd_new:
            cluster_corpus.append(item)
        
        
cluster_corpus[:20] 


['Thailand Reports No New Coronavirus Cases Or Deaths',
 'Coronavirus live updates: 6th death in Texas reported in Dallas County',
 'Malaysia reports 123 new coronavirus cases, total at 1,306',
 'Two new coronavirus deaths, as national total hits 37',
 'Stanford reports undergraduate student has coronavirus',
 'Coronavirus latest: Thailand confirms six more cases',
 'CDC reports 690,714 coronavirus cases, 35,443 deaths',
 'CDC reports 1,571,617 coronavirus cases, 94,150 deaths',
 'Malaysia reports 179 new coronavirus cases and 4 more deaths',
 'Thailand confirms fifth case of new coronavirus',
 'Malaysia, Philippines Report Hundreds Of New Coronavirus Cases',
 'Officials confirm 5 cases of coronavirus in Houston area, including 4 in Harris County',
 'Malaysia reports 54 new coronavirus cases, 2 new deaths',
 'Thailand reports 50 new coronavirus cases, 1 new death',
 'Latest travel advice for Thailand as coronavirus outbreak kills 132 people',
 'First 2 cases of coronavirus in Harris Co

In [333]:
Frame_corpus = pd.DataFrame(cluster_corpus, columns=['Frame_Sentence'])
Frame_corpus.to_csv('D://NLP//Frame_NLP//archive//Sent_Cluster//Frame86_Corpus.csv', index=False)

In [312]:
label_list_78 = Frames['Cluster 78'].tolist()
label_list_78[:50]

['coronavirus_market',
 'coronavirus_economy',
 'coronavirus_growth_economy_outbreak',
 'coronavirus_recession_economy',
 'says_depression_coronavirus_economy',
 'warns_recession_coronavirus',
 'crisis_economy_coronavirus_lockdown',
 'coronavirus_potential_case_economy',
 'lawmaker_coronavirus_economy_factor',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [322]:
SubFrame_corpus = {}


for i in label_list_78:
        if i != 'nan':
            SubFrame_sents = pd_all[pd_all['category']=='{}'.format(i)]['Processed_Title'].tolist()
            
        SubFrame_corpus = {**SubFrame_corpus, **{"{}".format(i): SubFrame_sents}}
        
            
    
    
Sub_Corpus = pd.DataFrame.from_dict(SubFrame_corpus, orient = 'index')
Sub_Corpus = Sub_Corpus.T

Sub_Corpus.head()



,coronavirus_market,coronavirus_economy,coronavirus_growth_economy_outbreak,coronavirus_recession_economy,says_depression_coronavirus_economy,warns_recession_coronavirus,crisis_economy_coronavirus_lockdown,coronavirus_potential_case_economy,lawmaker_coronavirus_economy_factor,nan
0,Coronavirus Fears Spread: What Are The Market Implications ?,Latin America faces 'lost decade' from coronavirus GDP drop,Coronavirus To Hurt Economic Growth In Many Countries: Moodys,Coronavirus and the latest indicators for the UK economy and society: 4 June 2020,Trump says US economy could take hit from coronavirus,Fed Chair Powell warns of prolonged U.S. recession after coronavirus,Coronavirus crisis: Lockdown batters India's already slowing economy,CORONAVIRUS: Best and worst case scenarios for global economy,Iranian Lawmaker: Hang Those Who Knowingly Spread Coronavirus,None
1,When The Coronavirus Outbreak Creates A Panic Buying Boom For Your Product,The economic fallout of coronavirus,Fed chairman: Coronavirus 'likely' to hinder economic growth in China,Coronavirus: Evening update as UK faces 'significant recession' BBC News,UPDATE 1 Trump says economy could take hit from coronavirus,"Coronavirus recession may already be here, market strategist says",Can extending India's lockdown stop coronavirus ?,GnS: Coronavirus Has The Potential To Trigger A Global Depression,"Iran's economy a factor in coronavirus response, President Rouhani says",None
2,European yields extend falls as investors assess coronavirus impact,Coronavirus: Atwoli in plea for rescue of economy,Coronavirus Outbreak Could Damage China's Economy: Sri Mulyani,Coronavirus and the latest indicators for the UK economy and society: 18 June 2020,Can Trump Avoid a Post Coronavirus Great Depression ?,Global recession risks have risen due to coronavirus Moody's,Coronavirus: Is it too early to ease lockdown restrictions in India ? DW 01.06.2020,Coronavirus recession: How bad it could get and what it means for you,Iran to use 20% of state budget to fight coronavirus,None
3,Commercial brokers say coronavirus not having a big impact yet,Coronavirus: Eight charts on how it has shaken economies,Wuhan coronavirus to have minimum effect on Ukraine's economy ministry,Singapore economy forecast to shrink 0.8% in 2020 from coronavirus hit: SP,Trump vows coronavirus ravaged economy will 'bounce' back,Recession Risks: Coronavirus + Oil Shock + What Else ?,India coronavirus lockdown day 6 ICMR denies community transmission,None,None,None
4,Covid 19 Coronavirus: NZ stockmarket dips back on the rollercoaster,"The Economic Hit From Coronavirus Will Be Worst Since The Great Depression, IMF Warns",Why Coronavirus outbreak in China is bad news for economy,UK economy shrinks 2% in first quarter on coronavirus,Trump says economy could take hit from coronavirus,Big banks bracing for coronavirus to cause first recession in 29 years,None,None,None,None


In [334]:
Sub_Corpus.to_csv('D://NLP//Frame_NLP//archive//Sent_Cluster//subframe78_Corpus.csv', index=False)
sub_frames = pd.read_csv('D://NLP//Frame_NLP//archive//Sent_Cluster//subframe78_Corpus.csv')

In [335]:
sub_frames.head()

,coronavirus_market,coronavirus_economy,coronavirus_growth_economy_outbreak,coronavirus_recession_economy,says_depression_coronavirus_economy,warns_recession_coronavirus,crisis_economy_coronavirus_lockdown,coronavirus_potential_case_economy,lawmaker_coronavirus_economy_factor,nan
0,Coronavirus Fears Spread: What Are The Market Implications ?,Latin America faces 'lost decade' from coronavirus GDP drop,Coronavirus To Hurt Economic Growth In Many Countries: Moodys,Coronavirus and the latest indicators for the UK economy and society: 4 June 2020,Trump says US economy could take hit from coronavirus,Fed Chair Powell warns of prolonged U.S. recession after coronavirus,Coronavirus crisis: Lockdown batters India's already slowing economy,CORONAVIRUS: Best and worst case scenarios for global economy,Iranian Lawmaker: Hang Those Who Knowingly Spread Coronavirus,NaN
1,When The Coronavirus Outbreak Creates A Panic Buying Boom For Your Product,The economic fallout of coronavirus,Fed chairman: Coronavirus 'likely' to hinder economic growth in China,Coronavirus: Evening update as UK faces 'significant recession' BBC News,UPDATE 1 Trump says economy could take hit from coronavirus,"Coronavirus recession may already be here, market strategist says",Can extending India's lockdown stop coronavirus ?,GnS: Coronavirus Has The Potential To Trigger A Global Depression,"Iran's economy a factor in coronavirus response, President Rouhani says",NaN
2,European yields extend falls as investors assess coronavirus impact,Coronavirus: Atwoli in plea for rescue of economy,Coronavirus Outbreak Could Damage China's Economy: Sri Mulyani,Coronavirus and the latest indicators for the UK economy and society: 18 June 2020,Can Trump Avoid a Post Coronavirus Great Depression ?,Global recession risks have risen due to coronavirus Moody's,Coronavirus: Is it too early to ease lockdown restrictions in India ? DW 01.06.2020,Coronavirus recession: How bad it could get and what it means for you,Iran to use 20% of state budget to fight coronavirus,NaN
3,Commercial brokers say coronavirus not having a big impact yet,Coronavirus: Eight charts on how it has shaken economies,Wuhan coronavirus to have minimum effect on Ukraine's economy ministry,Singapore economy forecast to shrink 0.8% in 2020 from coronavirus hit: SP,Trump vows coronavirus ravaged economy will 'bounce' back,Recession Risks: Coronavirus + Oil Shock + What Else ?,India coronavirus lockdown day 6 ICMR denies community transmission,NaN,NaN,NaN
4,Covid 19 Coronavirus: NZ stockmarket dips back on the rollercoaster,"The Economic Hit From Coronavirus Will Be Worst Since The Great Depression, IMF Warns",Why Coronavirus outbreak in China is bad news for economy,UK economy shrinks 2% in first quarter on coronavirus,Trump says economy could take hit from coronavirus,Big banks bracing for coronavirus to cause first recession in 29 years,NaN,NaN,NaN,NaN


## Construction Pattern Generator

In [ ]:
example_category = data_clustered[data_clustered['label_st1']==31].reset_index(drop=True)
example_category 

In [144]:
corpus = ['CDC reports 3,416,428 coronavirus cases',
 'U.S. coronavirus cases surpass 2.5 million: Reuters tally',
 'U.S. CDC reports 2,459,472 coronavirus cases',
 'U.S. CDC reports 4,024,492 coronavirus cases',
 'U.S. CDC reports 3,416,428 coronavirus cases',
 'U.S. surpasses 4 million confirmed coronavirus cases',
 'US hits 2.5M coronavirus cases as states tally record one day highs',
 "US' coronavirus tally crosses 2.5 million mark, toll at 125,480"]


In [72]:
corpus = pd_cluster[pd_cluster['category']=='postponed_anniversary_coronavirus_fear']['Processed_Title'].tolist()
corpus[:20]

['Cinderella Musical Postponed Due to Coronavirus Fears',
 "Coronavirus: Celebrations to mark 3rd anniversary of BJP govt in U'khand postponed",
 "'Bollywood Oscars' called off over coronavirus fears",
 'India EU summit postponed due to coronavirus',
 'Skysports awards suffers coronavirus postponement',
 'Oscars postponed due to coronavirus scourge',
 "Aamir Khan 'cancels' birthday celebrations amid coronavirus scare",
 'Coronavirus in India live updates: Attari Wagah border ceremony suspended',
 'LA Fashion Week delayed due to coronavirus',
 'Upcoming Padma Awards ceremony postponed in wake of coronavirus outbreak',
 'Peabody Awards Postponed Due to Coronavirus Pandemic',
 'Coronavirus in India Wagah beating retreat ceremony suspended',
 'Coronavirus: Celebrations to mark 3rd anniversary of BJP govt in Uttarakhand postponed',
 'Film academy delays 2021 Oscars ceremony because of coronavirus CBC.ca']

In [73]:
corpus_embeddings = embedder.encode(corpus)
# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(
    [corpus_embeddings[0]],
    corpus_embeddings[1:]
)
similarities

array([[0.43334538, 0.58618796, 0.5104109 , 0.4702193 , 0.66981226,
        0.36607003, 0.44138527, 0.5668489 , 0.5606787 , 0.6399028 ,
        0.44455236, 0.43352965, 0.58175826]], dtype=float32)

In [74]:
simi_list = similarities[0].tolist()
simi_list

[0.43334537744522095,
 0.5861879587173462,
 0.5104109048843384,
 0.47021931409835815,
 0.6698122620582581,
 0.366070032119751,
 0.44138526916503906,
 0.566848874092102,
 0.5606787204742432,
 0.6399027705192566,
 0.44455236196517944,
 0.43352964520454407,
 0.5817582607269287]

In [156]:
#example_doc = nlp(list(example_category['Processed_Title'])[1])

for i in corpus:
    example_doc = nlp(i)

    #print(f'{example_doc}\n')

    #for token in example_doc:
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_ , token.is_stop)

example_doc

US' coronavirus tally crosses 2.5 million mark, toll at 125,480

In [75]:
options = {"compact": True, "color":"green", "arrow_width":8, "arrow_spacing":12, 
           "word_spacing":15, "distance":150, "collapse_phrases":False}
simi_list = similarities[0].tolist()

for i in range(1):
    doc = nlp(corpus[i])
        
for i in range(1):
    print(corpus[i])
    displacy.render(doc, style="dep", options=options)

print('--------------------------------------------------------------------------')

for n in range(len(corpus)-1):
    i = simi_list[n-1]
    x = corpus[n+1]
    doc = nlp(x)
    

    print('{}\t\t{}'.format(x, i))
    displacy.render(doc, style="dep", options=options)
    print('--------------------------------------------------------------------------')
    

Cinderella Musical Postponed Due to Coronavirus Fears


--------------------------------------------------------------------------
Coronavirus: Celebrations to mark 3rd anniversary of BJP govt in U'khand postponed		0.5817582607269287


--------------------------------------------------------------------------
'Bollywood Oscars' called off over coronavirus fears		0.43334537744522095


--------------------------------------------------------------------------
India EU summit postponed due to coronavirus		0.5861879587173462


--------------------------------------------------------------------------
Skysports awards suffers coronavirus postponement		0.5104109048843384


...

--------------------------------------------------------------------------
Peabody Awards Postponed Due to Coronavirus Pandemic		0.5606787204742432


--------------------------------------------------------------------------
Coronavirus in India Wagah beating retreat ceremony suspended		0.6399027705192566


--------------------------------------------------------------------------
Coronavirus: Celebrations to mark 3rd anniversary of BJP govt in Uttarakhand postponed		0.44455236196517944


--------------------------------------------------------------------------
Film academy delays 2021 Oscars ceremony because of coronavirus CBC.ca		0.43352964520454407


--------------------------------------------------------------------------


In [65]:
options = {"compact": True, "color":"green", "arrow_width":8, "arrow_spacing":12, 
           "word_spacing":15, "distance":150, "collapse_phrases":False}
simi_list = similarities[0].tolist()

for i in range(1):
    doc = nlp(corpus[i])
        
for i in range(1):
    print(corpus[i])
    

print('--------------------------------------------------------------------------')

for n in range(len(corpus)-1):
    i = simi_list[n-1]
    x = corpus[n+1]
    doc = nlp(x)
    

    print('{}\t\t{}'.format(x, i))
    
    print('--------------------------------------------------------------------------')
    

Australian scientists grow copy of coronavirus in lab, called 'significant breakthrough'
--------------------------------------------------------------------------
Australian Scientists First to Grow Novel Coronavirus Outside China		0.6825590133666992
--------------------------------------------------------------------------
Global breakthrough as Australian lab grows coronavirus		0.8677197694778442
--------------------------------------------------------------------------
Aust grown coronavirus: why it's important		0.9021153450012207
--------------------------------------------------------------------------
'We got it': The game changing moment when this Australian lab recreated coronavirus		0.811951756477356
--------------------------------------------------------------------------
Aust scientists grow coronavirus in lab		0.8157276511192322
--------------------------------------------------------------------------
Scientists in Australia recreate coronavirus in potential 'game change

## Concept Entity Relationship Generator

## Noun Chuncks Extractor

## Frame Aspects Summarizer

## NER Tagger

## Lexical Units Extender

In [ ]:
# Word2Vec

## Most Frequent Words by POS

In [3]:
with open("D://NLP//Frame_NLP//archive//covid19_title.csv", encoding='utf8') as f:
    text = f.read().replace("\n\n", " ").replace("\n", " ")
doc = nlp(text)
sentences = list(doc.sents)
from collections import Counter
verbs = [ token.text for token in doc if token.is_stop != True and token.is_punct !=True and token.pos_ == 'VERB']
nouns = [ token.text for token in doc if token.is_stop != True and token.is_punct !=True and token.pos_ == 'NOUN']
adjs = [ token.text for token in doc if token.is_stop != True and token.is_punct !=True and token.pos_ == 'ADJ']
advs = [ token.text for token in doc if token.is_stop != True and token.is_punct !=True and token.pos_ == 'ADV']
print(Counter(verbs).most_common(20))
print(Counter(nouns).most_common(20))
print(Counter(adjs).most_common(20))
print(Counter(advs).most_common(20))

[('says', 42), ('confirms', 28), ('records', 24), ('tests', 22), ('fight', 19), ('dies', 14), ('rises', 13), ('hits', 11), ('announces', 11), ('reports', 11), ('confirmed', 11), ('urges', 10), ('stay', 10), ('warns', 9), ('reveals', 9), ('work', 9), ('reopen', 9), ('Says', 9), ('Help', 8), ('donates', 8)]
[('coronavirus', 186), ('cases', 122), ('outbreak', 53), ('patients', 35), ('lockdown', 29), ('death', 29), ('Cases', 27), ('deaths', 25), ('virus', 23), ('PM', 22), ('govt', 20), ('toll', 19), ('test', 19), ('pandemic', 17), ('hospital', 15), ('case', 14), ('masks', 14), ('state', 13), ('patient', 13), ('day', 13)]
[('new', 74), ('positive', 46), ('COVID19', 40), ('total', 21), ('social', 15), ('second', 11), ('old', 9), ('medical', 8), ('negative', 5), ('public', 5), ('low', 5), ('pandemic', 5), ('innovative', 4), ('single', 4), ('Spanish', 4), ('daily', 4), ('global', 4), ('Indian', 4), ('New', 4), ('federal', 4)]
[('far', 4), ('hard', 4), ('worldwide', 4), ('latest', 4), ('away', 

## Noun Chunks

In [11]:
chunks = (list(doc.noun_chunks))
for chunk in chunks:
    if "social" in str(chunk):
        print (chunk)

social distancing
'inappropriate' social media posts
social stock exchanges
social media
virus social distancing
social distancing
social distancing
social distancing
social distancing
federal social distancing guidelines
the last major social platforms
social distancing
just a social movement
